In [71]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import os
import torchvision.utils as vutils

In [72]:
matplotlib_is_available = True
try:
  from matplotlib import pyplot as plt
except ImportError:
  print("Will skip plotting; matplotlib is not available.")
  matplotlib_is_available = False

In [73]:
# Data params
data_mean = 1000
data_stddev = 100

In [74]:
# ### Uncomment only one of these to define what data is actually sent to the Discriminator
#(name, preprocess, d_input_func) = ("Raw data", lambda data: data, lambda x: x)
#(name, preprocess, d_input_func) = ("Data and variances", lambda data: decorate_with_diffs(data, 2.0), lambda x: x * 2)
#(name, preprocess, d_input_func) = ("Data and diffs", lambda data: decorate_with_diffs(data, 1.0), lambda x: x * 2)
(name, preprocess, d_input_func) = ("Only 4 moments", lambda data: get_moments(data), lambda x: 4)
print("Using data [%s]" % (name))

Using data [Only 4 moments]


In [75]:

def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))  # Gaussian

In [77]:

def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)  # Uniform-dist data into generator, _NOT_ Gaussian

In [78]:
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f

    def forward(self, x):
        x = self.map1(x)
        x = self.f(x)
        x = self.map2(x)
        x = self.f(x)
        x = self.map3(x)
        return x

In [79]:
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f

    def forward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        #return self.f(self.map3(x))
        return (self.map3(x))

In [80]:
def extract(v):
    return v.data.storage().tolist()


In [81]:
def stats(d):
    return [np.mean(d), np.std(d)]

In [82]:
def get_moments(d):
    # Return the first 4 moments of the data provided
    mean = torch.mean(d)
    diffs = d - mean
    var = torch.mean(torch.pow(diffs, 2.0))
    std = torch.pow(var, 0.5)
    zscores = diffs / std
    skews = torch.mean(torch.pow(zscores, 3.0))
    kurtoses = torch.mean(torch.pow(zscores, 4.0)) - 3.0  # excess kurtosis, should be 0 for Gaussian
    final = torch.cat((mean.reshape(1,), std.reshape(1,), skews.reshape(1,), kurtoses.reshape(1,)))
    return final

In [83]:
def decorate_with_diffs(data, exponent, remove_raw_data=False):
    mean = torch.mean(data.data, 1, keepdim=True)
    mean_broadcast = torch.mul(torch.ones(data.size()), mean.tolist()[0][0])
    diffs = torch.pow(data - Variable(mean_broadcast), exponent)
    if remove_raw_data:
        return torch.cat([diffs], 1)
    else:
        return torch.cat([data, diffs], 1)
    

In [93]:

# Model parameters

num_runs = 10

#job_name = "./saves/baseline/starter_gan_default"


# Model parameters
g_input_size = 1      # Random noise dimension coming into generator, per output vector
g_hidden_size = 5     # Generator complexity
g_output_size = 1     # Size of generated output vector
d_input_size = 500    # Minibatch size - cardinality of distributions
d_hidden_size = 10    # Discriminator complexity
d_output_size = 1     # Single dimension for 'real' vs. 'fake' classification
minibatch_size = d_input_size

d_learning_rate = 1e-3
g_learning_rate = 1e-3

num_epochs = 10000
print_interval = 100
save_interval = 10
d_steps = 5
g_steps = 1

dfe, dre, ge = 0, 0, 0
d_real_data, d_fake_data, g_fake_data = None, None, None

discriminator_activation_function = torch.sigmoid
generator_activation_function = torch.tanh



In [94]:
def plot_distribution(job_name, data_name, data):
    #print("Plotting the ", data_name, "...")
    #print(" Values: %s" % (str(gen_data)))
    fig = plt.hist(data, bins=50)
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.title('Histogram of ' + data_name + ' Distribution')
    plt.grid(True)
    plt.savefig(job_name + data_name + '.png')
    #plt.show()
    
   
    plt.clf()

In [95]:
def train(job_name):
    #print(job_name)
    d_sampler = get_distribution_sampler(data_mean, data_stddev)
    gi_sampler = get_generator_input_sampler()
    G = Generator(input_size=g_input_size,
                  hidden_size=g_hidden_size,
                  output_size=g_output_size,
                  f=generator_activation_function)
    D = Discriminator(input_size=d_input_func(d_input_size),
                      hidden_size=d_hidden_size,
                      output_size=d_output_size,
                      f=discriminator_activation_function)
    criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
    #d_optimizer = optim.SGD(D.parameters(), lr=d_learning_rate, momentum=sgd_momentum)
    #g_optimizer = optim.SGD(G.parameters(), lr=g_learning_rate, momentum=sgd_momentum)
    #d_optimizer = optim.Adam(D.parameters(), lr=d_learning_rate, betas=(beta1, 0.999))
    #g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=(beta1, 0.999))
    g_optimizer = optim.RMSprop(G.parameters(), lr=5e-5)
    d_optimizer = optim.RMSprop(D.parameters(), lr=5e-5)

    use_gpu = torch.cuda.is_available()
    #print("use_gpu: ", use_gpu)
    one = torch.FloatTensor([1]).cuda()
    mone = one * -1
    if use_gpu:
        one = one.cuda()
        mone = mone.cuda()
        G = G.cuda()
        D = D.cuda()
    start_time = time.time()
    if not os.path.exists(job_name + '_info_over_epoch.txt'):
        with open(job_name + '_info_over_epoch.txt', 'a'): pass
    open_file = open(job_name + '_info_over_epoch.txt', 'r+')
    open_file.seek(0)
    open_file.truncate()
    open_file.close()
    
    if not os.path.exists(job_name + '_results.txt'):
        with open(job_name + '_results.txt', 'a'): pass
    open_file = open(job_name + '_results.txt', 'r+')
    open_file.seek(0)
    open_file.truncate()
    open_file.close()
    
    avg_dfe = 0
    avg_dre = 0
    

    avg_ge = 0
    for epoch in range(num_epochs):
        
        for d_index in range(d_steps):
            # 1. Train D on real+fake
            D.zero_grad()

            #  1A: Train D on real
            d_real_data = Variable(d_sampler(d_input_size))
            
            
            
            if use_gpu:
                d_real_data = d_real_data.cuda()

            d_real_decision = D(preprocess(d_real_data))
            d_real_decision.backward(one)


            d_gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            if use_gpu:
                d_gen_input = d_gen_input.cuda()
            d_fake_data = G(d_gen_input).detach()  # detach to avoid training G on these labels
            d_fake_decision = D(preprocess(d_fake_data.transpose(0,1)))
            
            #Wassterstein loss
            #d_loss = -(torch.mean(d_real_decision) - torch.mean(d_fake_decision))
            d_fake_decision.backward(mone)
            errD = d_real_decision - d_fake_decision
            d_optimizer.step()     # Only optimizes D's parameters; changes based on stored gradients from backward()

            dre, dfe = d_real_decision.data[0], d_fake_decision.data[0]
            avg_dre += dre
            avg_dfe += dfe
            for p in D.parameters():
                p.data.clamp_(-0.01, 0.01)

            
            

        for g_index in range(g_steps):
            # 2. Train G on D's response (but DO NOT train D on these labels)
            G.zero_grad()

            gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            if use_gpu:
                gen_input = gen_input.cuda()
                              
            g_fake_data = G(gen_input)
            dg_fake_decision = D(preprocess(g_fake_data.transpose(0,1)))
                              
            
            #Wassterstein loss
            dg_fake_decision.backward(one)
            g_optimizer.step()  # Only optimizes G's parameters
            ge = dg_fake_decision.data[0]
            avg_ge += ge
        
       

        '''if epoch % print_interval == 0:
            print("Epoch %s: D (%s real_err, %s fake_err) G (%s err); Real Dist (%s),  Fake Dist (%s) " %
                  (epoch, dre, dfe, ge, stats(extract(d_real_data)), stats(extract(d_fake_data))))'''
        if epoch % save_interval == 0:
            with open(job_name + '_info_over_epoch.txt', 'a') as f:
                f.write("{} {} {} {}\n".format(epoch, avg_dre/(d_steps*save_interval), avg_dfe/(d_steps*save_interval), avg_ge/(g_steps*save_interval)))
            avg_dre = 0
            avg_dfe = 0
            avg_ge = 0

    #Plot real and generated figures
    if matplotlib_is_available:
        plot_distribution(job_name, 'Real', d_real_data.cpu())
        plot_distribution(job_name, 'Generated', extract(g_fake_data))
        
    #Calculate and save KL_DIV
    input_data = g_fake_data.detach().cpu()
    target_data = d_real_data.view(500, 1).cpu()
    min_elem = abs(min(torch.min(input_data), torch.min(target_data))) + 1
    #print(min_elem)
    #min_tensor = Variable(torch.Tensor([min_elem])).expand(input_data.size())
    #print(min_tensor)
    #print(min_tensor.size())

    kl_loss = torch.nn.functional.kl_div(torch.log(input_data + min_elem), target_data + min_elem)
    gen_mean = extract(torch.mean(input_data, dim = 0))
    gen_std = extract(torch.std(input_data, dim = 0))
    real_mean = extract(torch.mean(target_data, dim = 0))
    real_std = extract(torch.std(target_data, dim = 0))
    
    
    with open(job_name + '_results.txt', 'a') as f:
                f.write("""Generated Mean: {} \nGenerated StdDev: {} \nReal Mean: {} \nReal StdDev: {}\nKL Loss: {} \n""".format(gen_mean, gen_std, real_mean, real_std, kl_loss))
                f.close()
        
    torch.save({
                'model_state_dict': G.state_dict(),
                'optimizer_state_dict': g_optimizer.state_dict()
                }, job_name + '_generator_model')
    
    print("Total run time: ", time.time() - start_time)
    
    return gen_mean[0], gen_std[0], real_mean[0], real_std[0], kl_loss
    
    
    
        
        

In [96]:
def test_or_gen(start_train, model_name=None):
    if start_train:
        
        train()
    else:
        checkpoint=None
        if os.path.isfile(model_name + '_generator_model'):
            print("fetching model")
            checkpoint = torch.load(model_name + '_generator_model')
            G.load_state_dict(checkpoint['model_state_dict'])
            g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=(beta1, 0.999))
            g_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        gen_input = Variable(gi_sampler(minibatch_size, g_input_size)).cuda()
        g_fake_data = G(gen_input)
        print("Plotting the generated distribution...")
        gen_data = extract(g_fake_data)
        #print(" Values: %s" % (str(gen_data)))
        plt.hist(gen_data, bins=50)
        plt.xlabel('Value')
        plt.ylabel('Count')
        plt.title('Histogram of Generated Distribution')
        plt.grid(True)
        plt.show()


In [97]:
avg_gen_mean, avg_gen_std, avg_real_mean, avg_real_std, avg_kl_loss = 0, 0 ,0 ,0 ,0
path_name = "./saves/WL/WL_"
job_name = "/05_28_GAN_"
start_time = time.time()
for i in range(num_runs):
    new_path = path_name + str(i)
    os.mkdir(new_path)
    curr_job = new_path+job_name
    gen_mean, gen_std, real_mean, real_std, kl_loss = train(curr_job)
    print(f'--------- Run #{i} --------------')
    print(f"Generated Mean: {gen_mean}\nGenerated StdDev: {gen_std}\nReal_Mean: {real_mean}\nReal StdDev: {real_std}\nKL_Loss: {kl_loss}")
    
    avg_gen_mean += gen_mean
    avg_gen_std += gen_std
    avg_real_mean += real_mean
    avg_real_std += real_std
    avg_kl_loss += kl_loss
avg_gen_mean /= num_runs
avg_gen_std /= num_runs
avg_real_mean /= num_runs
avg_real_std /= num_runs
avg_kl_loss /= num_runs

final_data = "./saves/WL_final_results.txt"
if not os.path.exists(final_data):
    with open(final_data, 'a'): pass
open_file = open(final_data, 'r+')
open_file.seek(0)
open_file.truncate()
open_file.close()

with open(final_data, 'a') as f:
    f.write("""Generated Mean: {} \nGenerated StdDev: {} \nReal Mean: {} \nReal StdDev: {}\nKL Loss: {} \n""".format(avg_gen_mean, avg_gen_std, avg_real_mean, avg_real_std, avg_kl_loss))
    f.close()
print("Total run time: ", time.time() - start_time)


Total run time:  230.498920917511
--------- Run #0 --------------
Generated Mean: 0.042444054037332535
Generated StdDev: 0.0016914918087422848
Real_Mean: -0.1291152685880661
Real StdDev: 5.287949562072754
KL_Loss: 0.7949963808059692


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [46]:
'''checkpoint=None
if os.path.isfile(job_name + '_generator_model'):
    print("fetching model")
    checkpoint = torch.load(job_name + '_generator_model')
    G.load_state_dict(checkpoint['model_state_dict'])
    g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=(beta1, 0.999))
    g_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])'''

'''G.eval().cpu()
with torch.no_grad():
    gen_input = Variable(gi_sampler(minibatch_size, g_input_size))

    g_fake_data = G(gen_input)'''


#plot_distribution('Generated', extract(g_fake_data))

d_sampler = get_distribution_sampler(data_mean, data_stddev)

d_real_data = Variable(d_sampler(d_input_size))
x = torch.min(d_real_data)
print(x)



tensor(-20.2943)


In [19]:
test_sampler = get_distribution_sampler(100, 1)
input_1 = Variable(test_sampler(d_input_size))
input_2 = Variable(test_sampler(d_input_size))

kl_loss = torch.nn.functional.kl_div(torch.log(input_1), input_2)

print(kl_loss)


tensor(-0.0695)
